In [ ]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import pickle
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow.keras as keras
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import BatchNormalization


In [ ]:
img_width, img_height = 100, 100

base_model = ResNet50(weights= "imagenet",include_top = False , input_shape = (100,100,3))

for layer in base_model.layers:
    layer.trainable = True

In [ ]:
base_model.summary()

In [ ]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=BatchNormalization()(x)
x=Dense(1024,activation='tanh')(x) #dense layer 2
x=BatchNormalization()(x)
x=Dense(512,activation='relu')(x) #dense layer 3
x=BatchNormalization()(x)
x=Dense(256,activation='tanh')(x)
x=BatchNormalization()(x)
x=Dense(128,activation='relu')(x)
x=BatchNormalization()(x)
x=Dense(32,activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(0.5)(x)
preds=Dense(10,activation='softmax')(x) #final layer with softmax activation

In [ ]:
model=Model(inputs=base_model.input,outputs=preds)

adam = optimizers.Adam(learning_rate=0.001)

In [ ]:
train_path = "D:/GISA/DTDataset/"

In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,validation_split = 0.25) #included in our dependencies

train_generator=train_datagen.flow_from_directory(train_path,
                                                 target_size=(100,100),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 subset = 'training',
                                                 shuffle=True)

validation_generator = train_datagen.flow_from_directory(train_path,
                                                 target_size=(100,100),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 subset = 'validation',
                                                 shuffle=True)

In [ ]:
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
step_size_validation=validation_generator.n//validation_generator.batch_size
print(step_size_train)
model.fit_generator(generator=train_generator,
                    validation_data=validation_generator,
                   steps_per_epoch=step_size_train,
                    validation_steps=step_size_validation,
                   epochs=30)

model.save('Transfer-InceptionV3.model')
model.save_weights('Transfer-InceptionV3.h5')